# Introduction

# META Gradient Boosting Model

In this notebook I build a simple end-to-end ML pipeline on intraday stock data.

The goal is to:

- Download 15-minute OHLCV data for META from Yahoo Finance
- Engineer a rich set of technical features (RSI, EMA, ATR, Bollinger Bands, MACD, rolling stats, lags, simple regime feature)
- Define a forward-looking label that captures "good trades" based on a take-profit (TP) and stop-loss (SL) within a fixed horizon
- Train a Gradient Boosting classifier with time-series cross-validation
- Inspect feature importances and model quality
- Convert model probabilities into a very naive trading strategy and look at the resulting equity curve

This is **not** intended as a production-ready trading system.
The goal is to demonstrate how I approach:

- Feature engineering on time series
- Respecting temporal order (no leakage)
- Translating model outputs into something decision-oriented (in this case, a simple trading rule)

# Imports and Config

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import pandas_ta as ta

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
# Label definition: forward-looking horizon / TP / SL
# 15-minute bars → 4 bars/hour → 72h horizon = 72 * 4 = 288 bars
HORIZON_BARS = 288          # ~72 hours ahead on 15m data
TP = 0.015                  # +1.5% take profit
SL = -0.05                  # -5% max tolerated drawdown

# Trading system config
INITIAL_CAPITAL = 10_000
FEE_RT = 0.0024             # 0.24% round-trip fee
PROBA_THRESHOLD = 0.60      # Model probability threshold to take trade

# Regime feature config
REGIME_WINDOW = 200         # bars for regime_share rolling window (15m bars)